In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import os
from keras.utils import np_utils

In [6]:
IMG_SIZE = 100
training = []
x = [] #features
y = [] #labels
source = 'botheyes'
for filename in os.listdir(source):
    string = filename.split(".")
    img = cv2.imread(os.path.join(source,filename))
    x.append(img)
    y.append([int(string[0]), int(string[1])])
    training.append([img, [int(string[0]), int(string[1])]])
print(x)
print(y)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
x = np.array(x).reshape(len(x), 100*100*2*3)
x = x.astype('float32')
x /= 255
y = np.array(y)

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [15]:
x_train.shape

(3417, 60000)

In [16]:
y_train.shape

(3417, 2)

In [17]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

In [18]:
model = Sequential()
model.add(Dense(32, input_dim=60000, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(2, kernel_initializer='he_uniform'))
model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])

In [19]:
model.fit(x, y, verbose=0, epochs=100)

In [20]:
model.evaluate(x = x, y = y)

134/134 [==============================] - 17s 29ms/step - loss: 70.3195 - accuracy: 0.9239


[70.31949615478516, 0.9239231944084167]

In [4]:
model.predict(x_train[0])

NameError: name 'model' is not defined

In [21]:
predictions = model.predict(x_test)

In [22]:
predictions

array([[1189.4829  ,    8.702778],
       [ 117.443504,   55.790173],
       [  64.12434 ,  619.19855 ],
       ...,
       [ 315.7085  ,  321.36755 ],
       [ 253.08966 ,  316.98312 ],
       [ 375.78635 ,  767.09784 ]], dtype=float32)

In [23]:
y_test

array([[1216,   74],
       [  60,   89],
       [  51,  572],
       ...,
       [ 290,  376],
       [   8,  300],
       [ 228,  727]])

In [24]:
model_json = model.to_json()
with open("model_both.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model_both.h5")
print("Saved model to disk")

Saved model to disk


In [25]:
from keras.models import model_from_json

In [26]:
json_file = open('model_both.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("model_both.h5")
print("Loaded model from disk")

Loaded model from disk


In [1]:
from tkinter import * 
import threading
import random

In [2]:
from keras.models import model_from_json
from tkinter import * 
import threading
import pandas as pd
import numpy as np
import cv2
import os

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [5]:
x_predict = 0
y_predict = 0

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_eye.xml')

cap = cv2.VideoCapture(0)
_, img = cap.read()

def show_frame():
    global img, x_predict, y_predict
    while True:
        _, img = cap.read()
        faces = face_cascade.detectMultiScale(img, 1.3, 5)
        for (x,y,w,h) in faces:
            roi_gray = img[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray, 1.3, 5) # detect eyes
            width = np.size(roi_gray, 1) # get face frame width
            height = np.size(roi_gray, 0) # get face frame height
            left_eye_status = False
            right_eye_status = False
            for (x, y, w, h) in eyes:
                if y > height / 2:
                    pass
                eyecenter = x + w / 2  # get the eye center
                if eyecenter < width * 0.5:
                    left_eye = roi_gray[y:y + h, x:x + w]
                    left_eye = cv2.resize(left_eye, dsize=(100, 100))
                    left_eye_status = True
                if eyecenter > width * 0.5:
                    right_eye = roi_gray[y:y + h, x:x + w]
                    right_eye = cv2.resize(right_eye, dsize=(100, 100))
                    right_eye_status = True
            if left_eye_status and right_eye_status:
                eyes = cv2.hconcat([left_eye, right_eye])
                eyes = cv2.cvtColor(eyes, cv2.COLOR_BGR2GRAY)
                cv2.imshow("eye", eyes)
                eyes = np.array(eyes).reshape(-1, 30000)
                eyes = eyes.astype('float32')
                eyes /= 255
                prediction = loaded_model.predict([eyes])
                x_predict = prediction[0][0]
                y_predict = prediction[0][1]
        interrupt = cv2.waitKey(10)
        if interrupt & 0xFF == 27: # esc key
            break

def tkinter_frame():
    global myCanvas
    frame = Tk()  
    frame.attributes('-fullscreen',True) 
    
    myCanvas = Canvas(frame)
    myCanvas.pack(fill=BOTH, expand=True)
    
    frame.mainloop()

def pointer():
    global x_predict, y_predict
    while True:
        create_circle(x_predict, y_predict, 20)
        
def create_circle(x, y, r): #center coordinates, radius
    global myCanvas
    x0 = x - r
    y0 = y - r
    x1 = x + r
    y1 = y + r
    try:
        dot = myCanvas.create_oval(x0, y0, x1, y1, fill="red")  
    except:
        pass
        

t1 = threading.Thread(target=show_frame)
t2 = threading.Thread(target=tkinter_frame)
t3 = threading.Thread(target=pointer)

# starting thread 1
t1.start()
# starting thread 2
t2.start()
t3.start()

# wait until thread 1 is completely executed
# t1.join()
# wait until thread 2 is completely executed
t2.join() 

Exception in thread Thread-12:
Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\hp\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-22acf8245dae>", line 13, in show_frame
cv2.error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-q3d_8t8e\opencv\modules\videoio\src\cap_msmf.cpp:427: error: (-215:Assertion failed) videoSample in function '`anonymous-namespace'::SourceReaderCB::Wait'

